In [2]:
import sagemaker
import boto3
from sagemaker.amazon.amazon_estimator import get_image_uri
from sagemaker.session import s3_input, Session

In [3]:
bucket_name='bankapplication'
my_region=boto3.session.Session().region_name
print(my_region)

us-east-1


In [13]:
bucket_name='bankapplication9999'
s3=boto3.resource('s3')
try:
    if my_region == 'us-east-1':
        s3.create_bucket(Bucket=bucket_name)
        print('bucket created')
except Exception as e:
    print('error : ', e)
    


bucket created


In [14]:
prefix='xgboost-as-a-build-in-algo'
output_path='s3://{}/{}/output'.format(bucket_name,prefix)
print(output_path)

s3://bankapplication9999/xgboost-as-a-build-in-algo/output


In [15]:
import pandas as pd
import urllib


In [27]:
try:
    urllib.request.urlretrieve ("https://d1.awsstatic.com/tmt/build-train-deploy-machine-learning-model-sagemaker/bank_clean.27f01fbbdf43271788427f3682996ae29ceca05d.csv", "bank_clean.csv")
    print('download succesful')
except exception as e:
    print('error',e)
    
try:
    df=pd.read_csv('bank_clean.csv',index_col=0)
    print('read in panda')
except exception as e:
    print('error reading in pandas')
    
    

download succesful
read in panda


In [28]:
from sklearn.model_selection import train_test_split

In [31]:
y=df['y_yes']
X=df.drop(['y_yes','y_no'],axis=1)


In [32]:
X_train,X_test,y_train,y_test= train_test_split(X,y,test_size=.3, random_state=49)

In [37]:
df.shape , y_train.shape

((41188, 61), (28831,))

In [36]:
y_train.head()

17214    0
39221    1
10882    0
14835    0
33678    0
Name: y_yes, dtype: int64

In [42]:
import numpy as np

train,test=np.split(df.sample(frac=1,random_state=42), [int(0.8 * len(df))],axis=0)

In [51]:
import os
pd.concat([train['y_yes'], train.drop(['y_no', 'y_yes'], 
                                                axis=1)], 
                                                axis=1).to_csv('train.csv', index=False, header=False)
boto3.Session().resource('s3').Bucket(bucket_name).Object(os.path.join(prefix, 'train/train.csv')).upload_file('train.csv')
s3_input_train = sagemaker.TrainingInput(s3_data='s3://{}/{}/train'.format(bucket_name, prefix), content_type='csv')

In [53]:
pd.concat([test['y_yes'], test.drop(['y_no', 'y_yes'], axis=1)], axis=1).to_csv('test.csv', index=False, header=False)
boto3.Session().resource('s3').Bucket(bucket_name).Object(os.path.join(prefix, 'test/test.csv')).upload_file('test.csv')
s3_input_test = sagemaker.TrainingInput(s3_data='s3://{}/{}/test'.format(bucket_name, prefix), content_type='csv')

In [58]:
container = sagemaker.image_uris.retrieve('xgboost',boto3.Session().region_name,'1.2-2')

In [59]:
hyperparameters = {
        "max_depth":"5",
        "eta":"0.2",
        "gamma":"4",
        "min_child_weight":"6",
        "subsample":"0.7",
        "objective":"binary:logistic",
        "num_round":50
        }

In [63]:
estimator = sagemaker.estimator.Estimator(image_uri=container, 
                                          hyperparameters=hyperparameters,
                                          role=sagemaker.get_execution_role(),
                                          instance_count=1, 
                                          instance_type='ml.m5.2xlarge', 
                                          volume_size=5, # 5 GB 
                                          output_path=output_path)

In [64]:
estimator.fit({'train':s3_input_train,'validation':s3_input_test })

2022-05-10 12:59:40 Starting - Starting the training job...
2022-05-10 13:00:04 Starting - Preparing the instances for trainingProfilerReport-1652187580: InProgress
.........
2022-05-10 13:01:31 Downloading - Downloading input data...
2022-05-10 13:02:11 Training - Downloading the training image.....[2022-05-10 13:02:50.869 ip-10-0-132-248.ec2.internal:1 INFO utils.py:27] RULE_JOB_STOP_SIGNAL_FILENAME: None
[2022-05-10:13:02:50:INFO] Imported framework sagemaker_xgboost_container.training
[2022-05-10:13:02:50:INFO] Failed to parse hyperparameter objective value binary:logistic to Json.
Returning the value itself
[2022-05-10:13:02:50:INFO] No GPUs detected (normal if no gpus installed)
[2022-05-10:13:02:50:INFO] Running XGBoost Sagemaker in algorithm mode
[2022-05-10:13:02:50:INFO] Determined delimiter of CSV input is ','
[2022-05-10:13:02:50:INFO] Determined delimiter of CSV input is ','
[2022-05-10:13:02:50:INFO] Determined delimiter of CSV input is ','
[2022-05-10:13:02:51:INFO] Dete

In [67]:
xgb_predictor=estimator.deploy(initial_instance_count=1,instance_type='ml.m4.xlarge')

------!

In [70]:
test_array=test.drop(['y_yes','y_no'], axis=1).values


In [87]:
prediction = xgb_predictor.predict(test_array).decode('utf-8')

The csv_serializer has been renamed in sagemaker>=2.
See: https://sagemaker.readthedocs.io/en/stable/v2.html for details.


In [89]:
predictions_array = np.fromstring(prediction[1:], sep='\n') # and turn the prediction into an array


In [92]:
predictions_array.shape,test_array.shape

((8238,), (8238, 59))

In [94]:
cm = pd.crosstab(index=test['y_yes'], columns=np.round(predictions_array), rownames=['Observed'], colnames=['Predicted'])
tn = cm.iloc[0,0]; fn = cm.iloc[1,0]; tp = cm.iloc[1,1]; fp = cm.iloc[0,1]; p = (tp+tn)/(tp+tn+fp+fn)*100
print("\n{0:<20}{1:<4.1f}%\n".format("Overall Classification Rate: ", p))
print("{0:<15}{1:<15}{2:>8}".format("Predicted", "No Purchase", "Purchase"))
print("Observed")
print("{0:<15}{1:<2.0f}% ({2:<}){3:>6.0f}% ({4:<})".format("No Purchase", tn/(tn+fn)*100,tn, fp/(tp+fp)*100, fp))
print("{0:<16}{1:<1.0f}% ({2:<}){3:>7.0f}% ({4:<}) \n".format("Purchase", fn/(tn+fn)*100,fn, tp/(tp+fp)*100, tp))


Overall Classification Rate: 89.7%

Predicted      No Purchase    Purchase
Observed
No Purchase    91% (7188)    34% (102)
Purchase        9% (749)     66% (199) 



In [ ]:
sagemaker.Session().delete_endpoint(xgb_predictor.endpoint)
bucket_to_delete = boto3.resource('s3').Bucket(bucket_name)
bucket_to_delete.objects.all().delete()
